<div style="border: 2px solid #8A9AD0; margin: 1em 0.2em; padding: 0.5em;">

# SQL with R

by [The Carpentries](https://training.galaxyproject.org/hall-of-fame/carpentries/), [Helena Rasche](https://training.galaxyproject.org/hall-of-fame/hexylena/), [Avans Hogeschool](https://training.galaxyproject.org/hall-of-fame/avans-atgm/)

CC-BY licensed content from the [Galaxy Training Network](https://training.galaxyproject.org/)

**Objectives**

- How can I access databases from programs written in R?

**Objectives**

- Write short programs that execute SQL queries.
- Trace the execution of a program that contains an SQL query.
- Explain why most database applications are written in a general-purpose language rather than in SQL.

**Time Estimation: 45M**
</div>


<blockquote class="comment" style="border: 2px solid #ffecc1; margin: 1em 0.2em">
<h3 id="-icon-comment--comment">💬 Comment</h3>
<p>This tutorial is <strong>significantly</strong> based on <a href="https://carpentries.org">the Carpentries</a> <a href="https://github.com/swcarpentry/sql-novice-survey/">Databases and SQL</a> lesson, which is licensed CC-BY 4.0.</p>
<p>Abigail Cabunoc and Sheldon McKay (eds): “Software Carpentry: Using Databases and SQL.”  Version 2017.08, August 2017,
<a href="https://github.com/swcarpentry/sql-novice-survey">github.com/swcarpentry/sql-novice-survey</a>, <a href="https://doi.org/10.5281/zenodo.838776">https://doi.org/10.5281/zenodo.838776</a></p>
<p>Adaptations have been made to make this work better in a GTN/Galaxy environment.</p>
</blockquote>
<blockquote class="agenda" style="border: 2px solid #86D486;display: none; margin: 1em 0.2em">
<h3 id="agenda">Agenda</h3>
<p>In this tutorial, we will cover:</p>
</blockquote>
<p>For this tutorial we need to download a database that we will use for the queries.</p>


In [ ]:
download.file("http://swcarpentry.github.io/sql-novice-survey/files/survey.db", destfile="survey.db")

<h1 id="programming-with-databases---r">Programming with Databases - R</h1>
<p>Let’s have a look at how to access a database from
a data analysis language like R.
Other languages use almost exactly the same model:
library and function names may differ,
but the concepts are the same.</p>
<p>Here’s a short R program that selects latitudes and longitudes
from an SQLite database stored in a file called <code style="color: inherit">survey.db</code>:</p>


In [ ]:
library(RSQLite)
connection <- dbConnect(SQLite(), "survey.db")
results <- dbGetQuery(connection, "SELECT Site.lat, Site.long FROM Site;")
print(results)
dbDisconnect(connection)

<p>The program starts by importing the <code style="color: inherit">RSQLite</code> library.
If we were connecting to MySQL, DB2, or some other database,
we would import a different library,
but all of them provide the same functions,
so that the rest of our program does not have to change
(at least, not much)
if we switch from one database to another.</p>
<p>Line 2 establishes a connection to the database.
Since we’re using SQLite,
all we need to specify is the name of the database file.
Other systems may require us to provide a username and password as well.</p>
<p>On line 3, we retrieve the results from an SQL query.
It’s our job to make sure that SQL is properly formatted;
if it isn’t,
or if something goes wrong when it is being executed,
the database will report an error.
This result is a dataframe with one row for each entry and one column for each column in the database.</p>
<p>Finally, the last line closes our connection,
since the database can only keep a limited number of these open at one time.
Since establishing a connection takes time,
though,
we shouldn’t open a connection,
do one operation,
then close the connection,
only to reopen it a few microseconds later to do another operation.
Instead,
it’s normal to create one connection that stays open for the lifetime of the program.</p>
<p>Queries in real applications will often depend on values provided by users.
For example,
this function takes a user’s ID as a parameter and returns their name:</p>


In [ ]:
library(RSQLite)

connection <- dbConnect(SQLite(), "survey.db")

getName <- function(personID) {
  query <- paste0("SELECT personal || ' ' || family FROM Person WHERE id =='",
                  personID, "';")
  return(dbGetQuery(connection, query))
}

print(paste("full name for dyer:", getName('dyer')))

dbDisconnect(connection)

<p>We use string concatenation on the first line of this function
to construct a query containing the user ID we have been given.
This seems simple enough,
but what happens if someone gives us this string as input?</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">dyer'; DROP TABLE Survey; SELECT '
</code></pre></div></div>
<p>It looks like there’s garbage after the user’s ID,
but it is very carefully chosen garbage.
If we insert this string into our query,
the result is:</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">SELECT personal || ' ' || family FROM Person WHERE id='dyer'; DROP TABLE Survey; SELECT '';
</code></pre></div></div>
<p>If we execute this,
it will erase one of the tables in our database.</p>
<p>This is called an SQL injection attack,
and it has been used to attack thousands of programs over the years.
In particular,
many web sites that take data from users insert values directly into queries
without checking them carefully first.
A very <a href="https://xkcd.com/327/">relevant XKCD</a> that explains the
dangers of using raw input in queries a little more succinctly:</p>
<p><img src="https://imgs.xkcd.com/comics/exploits_of_a_mom.png" alt="A 4 panel comic, in the first panel a person is shown answering the phone, hearing that their son's school has some computer trouble. In panel 2 they apologises asking if their child broke something. In panel 3, the unseen person on the other end of the phone call asks if they really named their son Robert'); Drop table students;--? They respond saying 'oh yes. little bobby tables we call him.' In the 4th panel the caller says 'well we have lost this years student records, I hope you're happy.' They respond 'And I hope you've learned to sanitize your database inputs'." loading="lazy" /></p>
<p>Since an unscrupulous parent might try to smuggle commands into our queries in many different ways,
the safest way to deal with this threat is
to replace characters like quotes with their escaped equivalents,
so that we can safely put whatever the user gives us inside a string.
We can do this by using a prepared statement
instead of formatting our statements as strings.
Here’s what our example program looks like if we do this:</p>


In [ ]:
library(RSQLite)
connection <- dbConnect(SQLite(), "survey.db")

getName <- function(personID) {
  query <- "SELECT personal || ' ' || family FROM Person WHERE id == ?"
  return(dbGetPreparedQuery(connection, query, data.frame(personID)))
}

print(paste("full name for dyer:", getName('dyer')))

dbDisconnect(connection)

<p>The key changes are in the query string and the <code style="color: inherit">dbGetQuery</code> call (we use dbGetPreparedQuery instead).
Instead of formatting the query ourselves,
we put question marks in the query template where we want to insert values.
When we call <code style="color: inherit">dbGetPreparedQuery</code>,
we provide a dataframe
that contains as many values as there are question marks in the query.
The library matches values to question marks in order,
and translates any special characters in the values
into their escaped equivalents
so that they are safe to use.</p>
<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<h3 id="-icon-question--question-filling-a-table-vs-printing-values">❓ Question: Filling a Table vs. Printing Values</h3>
<p>Write an R program that creates a new database in a file called
<code style="color: inherit">original.db</code> containing a single table called <code style="color: inherit">Pressure</code>, with a
single field called <code style="color: inherit">reading</code>, and inserts 100,000 random numbers
between 10.0 and 25.0.  How long does it take this program to run?
How long does it take to run a program that simply writes those
random numbers to a file?</p>
</blockquote>
<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<h3 id="-icon-question--question-filtering-in-sql-vs-filtering-in-r">❓ Question: Filtering in SQL vs. Filtering in R</h3>
<p>Write an R program that creates a new database called
<code style="color: inherit">backup.db</code> with the same structure as <code style="color: inherit">original.db</code> and copies all
the values greater than 20.0 from <code style="color: inherit">original.db</code> to <code style="color: inherit">backup.db</code>.
Which is faster: filtering values in the query, or reading
everything into memory and filtering in R?</p>
</blockquote>
<h2 id="database-helper-functions-in-r">Database helper functions in R</h2>
<p>R’s database interface packages (like <code style="color: inherit">RSQLite</code>) all share
a common set of helper functions useful for exploring databases and
reading/writing entire tables at once.</p>
<p>To view all tables in a database, we can use <code style="color: inherit">dbListTables()</code>:</p>


In [ ]:
connection <- dbConnect(SQLite(), "survey.db")
dbListTables(connection)

<p>To view all column names of a table, use <code style="color: inherit">dbListFields()</code>:</p>


In [ ]:
dbListFields(connection, "Survey")

<p>To read an entire table as a dataframe, use <code style="color: inherit">dbReadTable()</code>:</p>


In [ ]:
dbReadTable(connection, "Person")

<p>Finally to write an entire table to a database, you can use <code style="color: inherit">dbWriteTable()</code>.
Note that we will always want to use the <code style="color: inherit">row.names = FALSE</code> argument or R
will write the row names as a separate column.
In this example we will write R’s built-in <code style="color: inherit">iris</code> dataset as a table in <code style="color: inherit">survey.db</code>.</p>


In [ ]:
dbWriteTable(connection, "iris", iris, row.names = FALSE)
head(dbReadTable(connection, "iris"))

<p>And as always, remember to close the database connection when done!</p>


In [ ]:
dbDisconnect(connection)

# Key Points

- Data analysis languages have libraries for accessing databases.
- To connect to a database, a program must use a library specific to that database manager.
- R's libraries can be used to directly query or read from a database.
- Programs can read query results in batches or all at once.
- Queries should be written using parameter substitution, not string formatting.
- R has multiple helper functions to make working with databases easier.

# Congratulations on successfully completing this tutorial!

Please [fill out the feedback on the GTN website](https://training.galaxyproject.org/training-material/topics/data-science/tutorials/sql-r/tutorial.html#feedback) and check there for further resources!
